# Feature selection
Per feature selection si intende un processo attraverso il quale restringere la dimensionalità degli elementi su insiemi di dati. Grazie ad esso è possibile ridurre l'overfitting, permettendo di prendere decisioni con meno "rumore", migliorare l'accuratezza e ridurre i tempi di training dell'algoritmo.  

In questo notebook, relativo al bike sharing nella città di Londra, analizzo i metodi di feature selection di sklearn. 

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LassoCV, Lasso

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
dataset = pd.read_csv('/kaggle/input/london-bike-sharing-dataset/london_merged.csv').drop(['timestamp', 't1', 't2'], axis=1) # Dropping columns with negative values
dataset.head()

**Unvariate selection (metodo Filter)**

Il primo metodo di feature selection è chiamato *unvariate selection*. Queste metodo utilizza test statistici per selezionare le caratteristiche che hanno relazioni più forti con la variabile di uscita.
La libreria scikit-learn fornisce la classe SelectKBest, che può essere utilizzata con una suite di test statistici diversi per selezionare un numero specifico di caratteristiche. In questo esempio utilizzo il test *Chi squared (Chi^2)* .

In [ ]:
dataset_values = dataset.values # Extract values from our dataset.

X = dataset_values[:,0:7] # Input
Y = dataset_values[:,6] # Target

test = SelectKBest(score_func=chi2, k=3) # Extract features, setting k value equal to 3. 
fit = test.fit(X, Y) 
print(fit.scores_) # Scores for each feature
features = fit.transform(X) # Apply the transformation
print(features[0:5,:])

Come possiamo vedere, l'*unvariate selection* con il test *Chi squared* estrae le tre migliori features del dataset (il parametro k è infatti impostato a 3), poiché presentano gli *scores* più alti.

**Recursive Feature Elimination - RFE (metodo Wrapper)**

Come è possibile intuire dall'acronimo, l'RFE consiste nel selezionare i dati considerando in modo ricorsivo insiemi sempre più piccoli di informazioni. 

In [ ]:
model = LogisticRegression(solver='lbfgs')
rfe = RFE(model, 3) # Select 3 features
fit = rfe.fit(X, Y)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

Come avviene la selezione di una feature?

In primo luogo, il predittore è allenato sull'insieme iniziale di elementi e l'importanza di ogni elemento è ottenuta sia attraverso un attributo *coef_* o attraverso un attributo *feature_importances_*. Quindi, le informazioni meno importanti vengono rimosse dall'insieme corrente, procedura che viene ripetuta ricorsivamente sull'insieme di elementi da selezionare fino a raggiungere il numero desiderato.

**Selection from model (metodo Embedded)**

I metodi Embedded sono metodi iterativi. Attraverso ogni iterazione del processo di formazione del modello, essi estraggono con attenzione quelle informazioni che contribuiscono maggiormente alla formazione per una particolare iterazione. I metodi di regolarizzazione sono i metodi embedded più comunemente usati che valutano un'informazione dato un coefficiente *treshold*.

In questo esempio viene utilizzata la regolarizzazione LASSO. Se l'informazione è irrilevante, il LASSO penalizza il suo coefficiente e lo rende 0. Tutto ciò con coefficiente = 0 viene rimosso mentre il resto viene preso. 

In [ ]:
reg = LassoCV()
reg.fit(X, Y)
print("Best alpha: %f" % reg.alpha_)
print("Best score: %f" %reg.score(X,Y))
coef = pd.Series(reg.coef_, index = list(dataset.columns.values))

In [ ]:
imp_coef = coef.sort_values()
matplotlib.rcParams['figure.figsize'] = (15.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso")

# Conclusioni
Come possiamo vedere, sono disponibili diversi metodi di feature selection, ognuno dei quali adatto per una specifica situazione. I metodi Filter utilizzano una *scoring function* per determinare l'utilità di un dato ma generalmente sono più imprecisi rispetto ai metodi Wrapper e Embedded. Questi ultimi, tuttavia, richiedono un tempo di esecuzione più lungo. Sono, quindi, più adatti quando il numero di features è ragionevolmente contenuto.  